# Data Dictionary

Build the data ditionary files

In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()



In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

pkg.set_sys_path()

from pylib.codebook import get_codebook_text

pkg

# General Social Survey
`norc.org-gss-2021-1.1.2` Last Update: 2023-03-06T16:40:56

_General Social Survey collects information and keeps a historical record of the concerns, experiences, attitudes, and practices of residents of the United States._


## Caveats

This package is an extract meant for a specific workflow. For general analysis, you should almost certainly use an extract generated from the [GSS website](https://gss.norc.org/). 

## Using the Data Files


The variables in this package are almost entirely categorical, but the data is stored with codes, with the categorical labels stored seperately in the ``_labels`` reference. When opening the file with Metapack, the ``convert_categorical`` argument ( which defaults to ``True`` ) controls converting the codes to labels. 

    pkg.resource('gss_2021').dataframe(convert_categorical=True)

Because the default is ``True`` this should happen automatically when using ``.dataframe``, but it is good to know what is happening behind the scenes.
## Documentation Links

* [General Social Survey](https://gss.norc.org/) GSS Homepage at NORC
* [Download page](https://gss.norc.org/get-the-data/stata) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[_labels](data/labels.csv)**. Variable value labels
* **[gss_2021](notebooks/extract.ipynb#df)**. General Social Survey 2021

## References
<ul><li> <strong><a href="https://gss.norc.org/Documents/stata/2021_stata.zip#GSS2021.dta">gss_2021_src</a></strong>. GSS 2021 Source  archive</li><li> <strong><a href="https://gss.norc.org/Documents/stata/2021_stata.zip#GSS 2021 Codebook.pdf">gss_2021_codes</a></strong>. GSS 2021 Codebook</li><li> <strong>_variables</string>, <em>data/variables.csv</em>. Variable names and descriptions</li><ul>

In [3]:
fn = pkg.reference('gss_2021_src').resolved_url.get_resource().get_target().fspath
df = pd.read_stata(fn)
df['year'] = df['year'].astype(int)
df.head()

/var/folders/kp/jn04x74j43j9v0t6cl0574h00000gn/T/ipykernel_15863/1164744907.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(fn)


,year,id,wrkstat,hrs1,hrs2,evwork,wrkslf,wrkgovt,occ10,prestg10,...,biblenv,postlifenv,kidssolnv,uscitznnv,fucitznnv,fepolnv,scibnftsv,abanyg,fileversion,vietdraft
0,2021,1,working full time,36.0,NaN,NaN,someone else,NaN,receptionists and information clerks,38.0,...,inspired word,no,somewhat worse,NaN,NaN,NaN,about equal (phone mode only: volunteered),no,7221.32,NaN
1,2021,2,working full time,45.0,NaN,NaN,someone else,NaN,advertising and promotions managers,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,benefits greater,NaN,7221.32,NaN
2,2021,3,working part time,16.0,NaN,NaN,someone else,NaN,miscellaneous assemblers and fabricators,35.0,...,NaN,NaN,NaN,NaN,NaN,disagree,NaN,yes,7221.32,NaN
3,2021,4,working part time,27.0,NaN,NaN,self-employed,NaN,childcare workers,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7221.32,NaN
4,2021,6,working full time,42.0,NaN,NaN,someone else,NaN,insurance claims and policy processing clerks,38.0,...,inspired word,yes,NaN,a u.s. citizen,NaN,disagree,NaN,yes,7221.32,NaN


In [4]:
# Get the variable descriptions out of the codebook

lines = get_codebook_text(pkg)


d = {}
pp = 0
state = None
for l in lines:
    l = l.strip()
    
    if l.startswith('Variable:'):
        state = 'var'
        _, var_name = l.split(': ')
       
        #if var_name == 'WIDOWED':
        #    pp = 30
    
        d[var_name] = ''
        
    elif l.startswith('Label:'):
        state = 'label'
        try:
            _, text = l.split(': ')
            d[var_name] += ' '+text.strip()
        except ValueError:
            pass
        except KeyError:
            #print('Err:', l)
            pass
                 
    elif l.startswith('LABEL') or l.startswith('VALUE COUNT') or l.startswith('Notes:') :
        state = None
        var_name = None
        
    elif state == 'label':
        d[var_name] += ' '+l.strip()
 
    if pp > 0:
        pp -= 1
        print(state, l)
        
variables_df = pd.DataFrame([ dict(column=k.lower(), desc=v) for k, v in d.items() ])
variables_df.head()

,column,desc
0,wrkstat,"Last week were you working full time, part ti..."
1,cohort,Birth cohort of respondent.
2,zodiac,ASTROLOGICAL SIGN OF RESPONDENT
3,hrs1,"IF WORKING, FULL OR PART TIME: How many hours..."
4,hrs2,"IF WITH A JOB, BUT NOT AT WORK: How many hour..."


In [5]:
from pandas.api.types import is_categorical_dtype

def cat_map(s):
    """ Produce a dict of categorical mappings. 
    The faster alternative, dict( enumerate(df[c].cat.categories ) ), misses the codes assocated with NaNs"""
    
    return dict( zip( s.cat.codes, s ) )

rows = []
for c in df.columns:
    s = df[c]
    
    # Convert identifier colums from categories to ints
    if is_categorical_dtype(s.dtype):
        if float(len(s.cat.categories)) / len(s) > .5:
            df[c] = df[c].cat.codes.astype(int)
        else:
            for k, v in cat_map(df[c]).items():
                rows.append({
                    'column': c,
                    'label': v, 
                    'code': k
                })
            
labels_df = pd.DataFrame(rows)
labels_df.head()
        

,column,label,code
0,wrkstat,working full time,0
1,wrkstat,working part time,1
2,wrkstat,retired,4
3,wrkstat,other,7
4,wrkstat,in school,5


In [8]:
variables_df

,column,desc
0,wrkstat,"Last week were you working full time, part ti..."
1,cohort,Birth cohort of respondent.
2,zodiac,ASTROLOGICAL SIGN OF RESPONDENT
3,hrs1,"IF WORKING, FULL OR PART TIME: How many hours..."
4,hrs2,"IF WITH A JOB, BUT NOT AT WORK: How many hour..."
...,...,...
997,sample,The GSS has employed various sampling procedu...
998,oversamp,Weight for Black oversamples.
999,phase,
1000,spaneng,Interviews Conducted in Spanish or English


In [ ]:
# Just saving the variables because we can. The data will always be accessed from the metadata
variables_df.to_csv('../data/variables.csv')
labels_df.to_csv('../data/labels.csv')

In [ ]:
# Update the column descriptions

descs = {r.column:r.desc for idx, r in variables_df.iterrows() }
changes = 0
st = pkg.resource('gss_2021').schema_term

for c in st.children:
    col_name = c.name
    
    desc_t = c.get_or_new_child('Description')
    
    if not desc_t.value:
        if desc_t.value != descs.get(col_name):
            change += 1
            desc_t.value = descs.get(col_name)
        
    
print(changes, 'changes')
if changes:
    pkg.write()


In [ ]:
variables_df

In [ ]:
labels_df.head()